# **`moma` tutorial**

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/deepmind/dm_robotics/blob/main/py/moma/moma_tutorial.ipynb)


> <p><small><small>Copyright 2021 The dm_robotics Authors.</small></p>
> <p><small><small>Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at <a href="http://www.apache.org/licenses/LICENSE-2.0">http://www.apache.org/licenses/LICENSE-2.0</a>.</small></small></p>
> <p><small><small>Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.</small></small></p>

# Imports

In [ ]:
#@title All `dm_robotics` imports required for this tutorial

# Agentflow provides useful utilities for MoMa.
from dm_robotics import agentflow as af
from dm_robotics.agentflow import spec_utils
from dm_robotics.agentflow.decorators import overrides
from dm_robotics.agentflow.preprocessors import observation_transforms
from dm_robotics.agentflow.preprocessors import rewards
from dm_robotics.agentflow.preprocessors import timestep_preprocessor
from dm_robotics.agentflow.subtasks import subtask_termination

# The geometry library helps with managing frames and sampling from
# random poses.
from dm_robotics.geometry import pose_distribution

# The MoMa modules used within this tutorial.
from dm_robotics.moma import action_spaces
from dm_robotics.moma import base_task
from dm_robotics.moma import effector
from dm_robotics.moma import entity_initializer
from dm_robotics.moma import prop
from dm_robotics.moma import robot
from dm_robotics.moma import sensor
from dm_robotics.moma import subtask_env_builder
from dm_robotics.moma.models import types
from dm_robotics.moma.models.arenas import empty
from dm_robotics.moma.models.end_effectors.robot_hands import robot_hand
from dm_robotics.moma.models.end_effectors.robot_hands import robotiq_2f85
from dm_robotics.moma.models.robots.robot_arms import robot_arm
from dm_robotics.moma.models.robots.robot_arms import sawyer
from dm_robotics.moma.models.robots.robot_arms import sawyer_constants
from dm_robotics.moma.sensors import prop_pose_sensor
from dm_robotics.moma.sensors import robot_arm_sensor
from dm_robotics.moma.sensors import robotiq_gripper_sensor
from dm_robotics.moma.tasks import run_loop

# The transformations library provides useful utils for converting between
# frames, from quaternions to euler angles, etc.
from dm_robotics.transformations import transformations

In [ ]:
#@title Other imports and helper functions

import enum
import os
from typing import Dict, Sequence, Tuple, Union

from dm_control import composer
from dm_control import mjcf
from dm_control import mujoco
from dm_control.composer.observation import observable
from dm_env import specs
from IPython.display import HTML
import matplotlib
import matplotlib.animation as animation
import matplotlib.pyplot as plt
import numpy as np
import PIL.Image
# To create and save animations, make sure FFmpeg is installed on your server.
# Check a path to FFmpeg binary with `which ffmpeg`, and use the output
# in the notebook, for example:
# matplotlib.rcParams['animation.ffmpeg_path'] = '/usr/bin/ffmpeg`


def render_scene(physics: mjcf.Physics) -> np.ndarray:
  camera = mujoco.MovableCamera(physics, height=360, width=480)
  camera.set_pose([0, 0, 0], 2.5, 180, -35)
  return camera.render()

def display_images(images, titles):
  assert len(images) == len(titles)
  fig, axs = plt.subplots(1, len(images))
  for ax, img, title in zip(axs, images, titles):
    ax.imshow(img)
    ax.axes.xaxis.set_visible(False)
    ax.axes.yaxis.set_visible(False)
    ax.set_title(title)
  fig.set_size_inches(12, 12)

# Inline video helper function
if os.environ.get('COLAB_NOTEBOOK_TEST', False):
  # We skip video generation during tests, as it is quite expensive.
  display_video = lambda *args, **kwargs: None
else:
  def display_video(frames, framerate=30):
    height, width, _ = frames[0].shape
    dpi = 70
    orig_backend = matplotlib.get_backend()
    matplotlib.use('Agg')  # Switch to headless 'Agg' to inhibit figure rendering.
    fig, ax = plt.subplots(1, 1, figsize=(width / dpi, height / dpi), dpi=dpi)
    matplotlib.use(orig_backend)  # Switch back to the original backend.
    ax.set_axis_off()
    ax.set_aspect('equal')
    ax.set_position([0, 0, 1, 1])
    im = ax.imshow(frames[0])
    def update(frame):
      im.set_data(frame)
      return [im]
    interval = 1000/framerate
    anim = animation.FuncAnimation(fig=fig, func=update, frames=frames,
                                   interval=interval, blit=True, repeat=False)
    return HTML(anim.to_html5_video())

# Introduction

MoMa (Modular Manipulation) is DeepMind's library for building robotic manipulation tasks for reinforcement learning (RL). In this tutorial, we will run through important MoMa concepts while building a "lift-a-block" task with a Sawyer robot arm, a Robotiq parallel gripper, and some objects.

For a more complex example, see `tasks/example_task/` in the project folder.

## The `dm_env` Environment interface

In RL tasks, there is typically an "agent" and an "environment", where the agent provides actions and the environment responds with observations and rewards. There are many ways to codify that agent-environment interaction, and within this tutorial, we will use the [`dm_env.Environment`](https://github.com/deepmind/dm_env) interface where the agent provides actions through a list of floats and the environment responds with a `dm_env.TimeStep` consisting of observations and rewards.

Note that MoMa is not tied to this environment interface. For other, more complex uses of MoMa, see the `dm_robotics` Agentflow module.

# Build the task

There are 3 main components of every MoMa task: the MuJoCo model of the scene, the hardware abstraction, and the task-specific logic.

For this tutorial, we will work with a single Sawyer arm, a Robotiq parallel gripper, and a block in simulation  to build a "lift-a-block" task.

## Build the scene

MoMa is built on top of Composer, which is part of [`dm_control`](https://github.com/deepmind/dm_control), a DeepMind library for managing MuJoCo simulation environments.

Before we can start writing our task, we need to build the scene in simulation, which we can do using Composer tools.

**NOTE**: Even if you are creating a task for real robots (which MoMa supports as well!), you must do this step. MoMa requires a working MuJoCo model of the scene for both simulation and real tasks. The MuJoCo model is more useful in the simulation case, but even for real-world environments, it is used to help with computing observations and IK.

In [ ]:
def _build_arena(name: str) -> composer.Arena:
  """Returns an empty arena where we can put everything."""
  arena = empty.Arena(name)
  arena.mjcf_model.option.timestep = 0.001
  arena.mjcf_model.option.gravity = (0., 0., -1.0)
  arena.mjcf_model.size.nconmax = 1000
  arena.mjcf_model.size.njmax = 2000
  arena.mjcf_model.visual.__getattr__('global').offheight = 480
  arena.mjcf_model.visual.__getattr__('global').offwidth = 640
  arena.mjcf_model.visual.map.znear = 0.0005
  return arena

def _add_robot_and_gripper(
    arena: composer.Arena) -> Tuple[composer.Entity, composer.Entity]:
  arm = sawyer.Sawyer(
      name='sawyer', actuation=sawyer_constants.Actuation.INTEGRATED_VELOCITY)
  gripper = robotiq_2f85.Robotiq2F85()
  # Attach the gripper to the arm.
  robot.standard_compose(arm=arm, gripper=gripper)
  # And attach arm to the arena.
  arena.attach(arm)
  return arm, gripper

def _add_block(arena: composer.Arena) -> composer.Entity:
  block = prop.Block()
  frame = arena.add_free_entity(block)
  block.set_freejoint(frame.freejoint)
  return block

# Build the scene.
arena = _build_arena('sawyer_with_block')
arm, gripper = _add_robot_and_gripper(arena)
block = _add_block(arena)

# Visualize it.
physics = mjcf.Physics.from_mjcf_model(arena.mjcf_model)
PIL.Image.fromarray(render_scene(physics))

In the visualization above, you can see the Sawyer arm with the Robotiq gripper. The block may not be visible, but we will fix that once we get to the initializers section below.

## Hardware abstraction

In MoMa, we abstract away the physical components in an environment with 2 classes: `sensor.Sensor` and `effector.Effector`:

```python
from dm_robotics.moma import effector
from dm_robotics.moma import sensor
```

A `Sensor` produces observations and an `Effector` consumes actions and actuates parts of the environment.

Now, let's create some sensors and effectors for the physical setup we have.

### Effectors

Let's control the Sawyer with joint velocity commands, and let's control the parallel gripper with 1D velocity commands.

We already have several basic effectors already available in MoMa that provide joint control, Cartesian control, etc. But for this tutorial, we will implement the joint velocity effector from scratch.

For existing effectors, see `moma/effectors/`.

Every MoMa effector simply needs to extend the abstract base class in `effector.py` in order to (a) give a spec for the expected commands to that effector and (b) define how it consumes commands that adhere to its spec.

In [ ]:
class JointVelocityEffector(effector.Effector):
  """Actuate all the velocity MuJoCo actuators in an arm or gripper."""

  def __init__(self,
               entity: Union[robot_arm.RobotArm, robot_hand.RobotHand],
               name: str):
    """JointVelocityEffector constructor.

    Args:
      entity: A robot arm or gripper with an `actuators` property. This class
        assumes that the actuators are all in velocity or integrated velocity
        mode.
      name: Name of the hardware being controlled.
    """
    self._entity = entity
    self._name = name
    self._action_spec: specs.BoundedArray = None

  def initialize_episode(self, physics: mjcf.Physics,
                         random_state: np.random.RandomState) -> None:
    """Called before each episode."""
    pass  # This effector is not stateful, so no initialization is needed.

  def action_spec(self, physics: mjcf.Physics) -> specs.BoundedArray:
    """Returns details for what kind of command is expected by set_control."""
    if self._action_spec is None:
      num_actuators = len(self._entity.actuators)
      actuator_names = [f'{self.prefix}{i}'
                        for i in range(num_actuators)]
      action_min, action_max = self._action_range_from_actuators(
          physics, self._entity.actuators)
      self._action_spec = specs.BoundedArray(
          shape=(num_actuators,),
          dtype=np.float32,
          minimum=action_min,
          maximum=action_max,
          name='\t'.join(actuator_names))
    return self._action_spec

  def set_control(self, phys: mjcf.Physics, command: np.ndarray) -> None:
    """Actuates all the actuators in the entity."""
    spec_utils.validate(self.action_spec(phys), command)
    phys.bind(self._entity.actuators).ctrl = command

  @property
  def prefix(self) -> str:
    return f'{self._name}_actuator'

  def _action_range_from_actuators(
      self,
      physics: mjcf.Physics,
      actuators: Sequence[types.MjcfElement]) -> Tuple[np.ndarray, np.ndarray]:
    """Returns the action range min, max for the actuators."""
    num_actions = len(actuators)
    control_range = physics.bind(actuators).ctrlrange
    is_limited = physics.bind(actuators).ctrllimited.astype(np.bool)
    minima = np.full(num_actions, fill_value=-np.inf, dtype=np.float32)
    maxima = np.full(num_actions, fill_value=np.inf, dtype=np.float32)
    minima[is_limited], maxima[is_limited] = control_range[is_limited].T
    return minima, maxima

# Create effectors for the arm and gripper.
# (minor note: the actuators in a Robotiq gripper are not technically
# joint-level because of coupling in Robotiq 2F85 grippers, but that doesn't
# practically affect what we are doing here.)
arm_joint_effector = JointVelocityEffector(entity=arm, name='sawyer')
gripper_effector = JointVelocityEffector(entity=gripper, name='robotiq')

In [ ]:
# Visualize the starting state.
before = render_scene(physics)

# Try running the effectors in a loop.
num_steps = int(1. / physics.timestep())  # run for a second
arm_command = np.ones_like(
    arm_joint_effector.action_spec(physics).minimum) * 0.5
gripper_command = np.ones_like(
    gripper_effector.action_spec(physics).minimum) * 0.5
for _ in range(num_steps):
  arm_joint_effector.set_control(physics, arm_command)
  gripper_effector.set_control(physics, gripper_command)
  physics.step()

# Visualize the new state of things.
after = render_scene(physics)

display_images([before, after], ['Before actuation', 'After actuation'])

### Sensors

Sensors in MoMa extend the abstract base class defined in `sensor.py`. They provide observations via Composer observables. See `dm_control.composer` for more info on the observable API.

We have several existing sensor implementations in `moma/sensors/`. In this tutorial, let's implement a simple sensor that will give us the pose of the gripper in the world-frame.

In [ ]:
@enum.unique
class GripperPoseObservations(enum.Enum):
  """Observations exposed by this sensor.
  
  Typically, for each MoMa sensor we provide a corresponding enum that tracks
  all the available observations for the sensor. It helps with programatically
  fetching observations later when we get to the RL environment.
  """
  # The world x,y,z position of the gripper's tool-center-point.
  POS = '{}_pos'
  # The world orientation quaternion of the gripper's tool-center-point.
  QUAT = '{}_quat'

  def get_obs_key(self, name: str) -> str:
    """Returns the key to the observation in the observables dict."""
    return self.value.format(name)

class GripperPoseSensor(sensor.Sensor[GripperPoseObservations]):

  def __init__(self, gripper: robot_hand.RobotHand, name: str):
    self._gripper = gripper
    self._name = name

    # Mapping of observation names to the composer observables (callables that
    # will produce numpy arrays containing the observations).
    self._observables = {
        self.get_obs_key(GripperPoseObservations.POS):
            observable.Generic(self._pos),
        self.get_obs_key(GripperPoseObservations.QUAT):
            observable.Generic(self._quat),
    }
    for obs in self._observables.values():
      obs.enabled = True

  def initialize_episode(self, physics: mjcf.Physics,
                         random_state: np.random.RandomState) -> None:
    pass  # Nothing special needed each episode.

  @property
  def observables(self) -> Dict[str, observable.Observable]:
    return self._observables

  @property
  def name(self) -> str:
    return self._name

  def get_obs_key(self, obs: GripperPoseObservations) -> str:
    return obs.get_obs_key(self._name)

  def _pos(self, physics: mjcf.Physics) -> np.ndarray:
    return physics.bind(self._gripper.tool_center_point).xpos

  def _quat(self, physics: mjcf.Physics) -> np.ndarray:
    rmat = physics.bind(self._gripper.tool_center_point).xmat
    quat = transformations.mat_to_quat(np.reshape(rmat, [3, 3]))
    return transformations.positive_leading_quat(quat)

In [ ]:
# Create a sensor for the gripper.
gripper_pose_sensor = GripperPoseSensor(gripper, 'gripper')

# Check out the poses returned by the sensor.
for observation_name, obs_callable in gripper_pose_sensor.observables.items():
  print(f'{observation_name}: {obs_callable(physics)}')

### Switching between simulated and real robots.

The `Effector` and `Sensor` classes are usable by both simulated entities _and_ real-world entities. For instance, the velocity effector we wrote above can easily be replaced by an `Effector` that publishes joint velocity commands via ROS to a physical robot.

The rest of this tutorial will cover the rest of the task logic we can configure within MoMa, but it is important to note that, in order to move from a simulated task to a real-world one, all you need to do is change the sensors and effectors to their real-world equivalents. The rest of MoMa is agnostic to sim vs. real.

As noted above, even for real robot environments, you still need to provide a MuJoCo model of the scene.

## MoMa `BaseTask`

The MuJoCo scene, sensors, and effectors represent the basis of any task we want to build with this physical setup. We could create a task where the arm needs to reach the block or where it needs to pick-and-place the block to a new location. We could change the types of observations we want to actually surface to the agent or add new observations (like the distance from the gripper to the block).

All of those extra bits are task-specific, but the physical basis remains constant. In MoMa, we wrap that basis with a class `BaseTask` (see `base_task.py`).

`BaseTask` is the simplest possible task we could give to an agent. It provides a zero reward, an empty action spec (the agent can't do anything), and all the possible observations from all the sensors registered. In later sections, we will discuss how to build on top of `BaseTask` to make more interesting tasks.

In [ ]:
# Rather than deal with the raw Compose entities, we wrap the arms and grippers,
# along with their relevant effectors and sensors, with a `Robot` abstraction.
robot_sensors = [
    # Sensor for the joint angles, velocities, and torques.
    robot_arm_sensor.RobotArmSensor(
        arm=arm, name='sawyer', have_torque_sensors=True),
    # Sensor for the gripper position in the world frame (see prev section.)
    gripper_pose_sensor,
    # Sensor for the gripper state (width of the fingers).
    robotiq_gripper_sensor.RobotiqGripperSensor(gripper, name='gripper_state'),
]
sawyer_and_gripper = robot.StandardRobot(
    arm=arm,
    arm_base_site_name='pedestal_attachment',
    gripper=gripper,
    robot_sensors=robot_sensors,
    arm_effector=arm_joint_effector,
    gripper_effector=gripper_effector)

# Keep track of the pose of the block, in the world frame.
block_pose_sensor = prop_pose_sensor.PropPoseSensor(block, name='block')

task = base_task.BaseTask(
    task_name='sawyer_lift_block',
    arena=arena,
    robots=[sawyer_and_gripper],
    props=[block],
    extra_sensors=[block_pose_sensor],
    extra_effectors=[],  # Add any effectors not associated with a robot here.
    control_timestep=0.1,  # Run the task at 10 Hz.
    # We will revisit the initializes in a later section.
    scene_initializer=lambda _: None,
    episode_initializer=lambda _1, _2: None)

## Task logic

Through the scene and hardware abstraction, we've built the foundation to design any task we want with a Sawyer, a gripper, and a single block. Now, we need to answer 3 questions:

1. What action space do we want to expose to our agents?
2. How do we want to initialize our episodes?
3. What rewards, episode termination criteria, etc. do we want?

This seciton runs through those 3 questions for our task: lifting the block with the arm and gripper.

### Action spaces

MoMa borrows the concept of an action space from DeepMind's Agentflow. An `af.ActionSpace` projects an action (a Numpy array) to a new space (another Numpy array).

You can find commonly used `af.ActionSpace` subclasses within `action_spaces.py`. Important note: in practice, we actually do a lot of the projecting from one action space to another inside the `Effector` classes. For instance, the `cartesian_6d_velocity_effector.py` projects 6D Cartesian velocity commands to joint space, and then delegates those commands to an underlying effector it wraps. However, we could do that via an `af.ActionSpace` instead.

For this tutorial, let's not worry about projecting action spaces, but rather, let's combine action spaces via the `af.CompositeActionSpace`.

In [ ]:
# First get the complete action spec (not space!) from the base task.
parent_action_spec = task.effectors_action_spec(physics)

# Then create an action space associated with each effector.
# Joint space control for the arm.
joint_action_space = action_spaces.ArmJointActionSpace(
    af.prefix_slicer(parent_action_spec, arm_joint_effector.prefix))
# 1D control for the gripper.
gripper_action_space = action_spaces.GripperActionSpace(
    af.prefix_slicer(parent_action_spec, gripper_effector.prefix))

# If we didn't want to give raw control to all the joints of the arm, or if we
# wanted to remove control of the gripper, we could add such logic here and
# modify the action space. However, for this task, we will keep it simple and
# just use each action space as is.

# Combine them into a composite action space, which we can expose to the agents.
combined_action_space = af.CompositeActionSpace(
    [joint_action_space, gripper_action_space])

# Check the spec of the action space, and you should see 8 actions: 7 for the
# arm's joints and 1 for the gripper.
combined_action_space.spec()

Note: Even though the action space defined above is fairly trivial, it is necessary for building the RL environment. Remember that the `BaseTask` we defined above has a **empty** action spec, meaning the agent cannot give any actions. We will use `combined_action_space` later to allow the RL agent to provide non-empty actions.

### Initializing episodes

In RL, the agent-environment interactions are often split up into "episodes" with a start and end. Before the start of each episode, we often will want to reset the environment and put entities in randomized locations. In MoMa, we have a set of scene and episode initializers to help do that.

A `SceneInitializer` is a callable that changes the MJCF (MuJoCo model) of the scene. Usually this entails moving entities around and changing their attachment sites or fixed joint positions, but it could also involve changing friction, lighting, etc. for domain randomization. It is typically not used for real-world environments.

An episode initializer is a `composer.Initializer` that is run after the `SceneInitializer` is run and the MJCF is recompiled. You can find a few useful initializers within `entity_initializer.py`.

Scene initialization is outside the scope of this tutorial. Instead, we will focus on initializing 2 things: the joints of the arm and the position of the block.

In [ ]:
# To initialize the arm joints in a useful starting position, let's:
#   1. Sample a random goal pose for the gripper within some 3D bounding box.
#   2. Do IK to calculate the arm joint angles needed to place the gripper at
#      that goal pose.
# Luckily, there are out-of-the-box MoMa utilities we can use to do this.
gripper_pose_dist = pose_distribution.UniformPoseDistribution(
    # Provide 6D min and max bounds for the 3D position and 3D euler angles.
    min_pose_bounds=np.array([0.5, -0.1, 0.1,
                              0.75 * np.pi, -0.25 * np.pi, -0.5 * np.pi]),
    max_pose_bounds=np.array([0.7, 0.1, 0.2,
                              1.25 * np.pi, 0.25 * np.pi, 0.5 * np.pi]))
initialize_arm = entity_initializer.PoseInitializer(
    # `position_gripper()` is a function that performs the IK and moves the
    # joints of the arm to the input gripper pose.
    sawyer_and_gripper.position_gripper, gripper_pose_dist.sample_pose)

# To initialize the block, we can run through a similar process of sampling a
# pose and then moving the block to that pose.
block_pose_dist = gripper_pose_dist = pose_distribution.UniformPoseDistribution(
    # Provide 6D min and max bounds for the 3D position and 3D euler angles.
    min_pose_bounds=np.array([0.5, -0.1, 0.05, 0.0, 0.0, -np.pi]),
    max_pose_bounds=np.array([0.7, 0.1, 0.05, 0.0, 0.0, np.pi]))
initialize_block = entity_initializer.PoseInitializer(
    block.set_pose, pose_sampler=block_pose_dist.sample_pose)

# Combine the 2 initializers so they both run.
entities_initializer = entity_initializer.TaskEntitiesInitializer(
    [initialize_arm, initialize_block])

In [ ]:
# Run the initializer and see what the scene looks like.
before = render_scene(physics)
entities_initializer(physics, np.random.RandomState())
physics.step()  # propogate the changes from the initializer.
after = render_scene(physics)

# Visualize the new state of things.
display_images([before, after], ['Before init', 'After init'])

Notice that you can see the block now! The gripper should be pointing somewhat downwards above the block (not directly above necessarily). Feel free to change the random state and see more poses the entities can end up in.

### Timestep preprocessors

Timestep preprocessors are a powerful tool MoMa borrows from DeepMind's Agentflow. An environment timestep is a collection of the rewards and observations coming from the environment. The timestep preprocessors modify the timesteps coming from the `BaseTask` before giving them to the agent.

Within the timestep preprocessors, we can do many things including:
 - Add/remove/modify observations
 - Add rewards
 - Terminate an episode

These define the task-specific logic of an RL environment. For instance, to change an environment from a lift-a-block task to a pick-and-place task, all you need to do is change the reward function in the timestep preprocessors.

You can find many useful timestep preprocessors in `dm_robotics.agentflow.preprocessors`. 

For our task, let's some existing timestep preprocessors and define a new one as well.

In [ ]:
preprocessors = []

# Make sure everything has the same dtype (simplifies some logic down the line).
preprocessors.append(observation_transforms.CastPreprocessor(dtype=np.float32))

# Let's add a new observation that gives a vector from the gripper position to
# the block position, which could help the agent learn to reach the block.
# Note that this is possible via observation_transforms.AddObservation as well,
# but we'll implement it here just to show the TimestepPreprocessor API.
class AddGripperToBlockObservation(timestep_preprocessor.TimestepPreprocessor):

  # Name of the observation this adds.
  OBS_KEY: str = 'gripper_to_block'

  def __init__(self, gripper_obs: str, block_obs: str):
    self._gripper_obs = gripper_obs
    self._block_obs = block_obs
    super().__init__()

  @overrides(timestep_preprocessor.TimestepPreprocessor)
  def _process_impl(
      self, timestep: timestep_preprocessor.PreprocessorTimestep
      ) -> timestep_preprocessor.PreprocessorTimestep:
    gripper = timestep.observation[self._gripper_obs]
    block = timestep.observation[self._block_obs]
    timestep.observation[AddGripperToBlockObservation.OBS_KEY] = (
        gripper[:3] - block[:3])
    return timestep

  @overrides(timestep_preprocessor.TimestepPreprocessor)
  def _output_spec(
      self, input_spec: spec_utils.TimeStepSpec) -> spec_utils.TimeStepSpec:
    obs_spec = dict(input_spec.observation_spec)
    obs_spec[AddGripperToBlockObservation.OBS_KEY] = (
        specs.Array(shape=(3,), dtype=np.float32))
    return input_spec.replace(observation_spec=obs_spec)

preprocessors.append(
    AddGripperToBlockObservation(
        gripper_pose_sensor.get_obs_key(GripperPoseObservations.POS),
        block_pose_sensor.get_obs_key(prop_pose_sensor.Observations.POSE)))


# Let's only expose a subset of the observations coming from the sensors to the
# agent.
preprocessors.append(
    observation_transforms.RetainObservations([
        'sawyer_joint_pos',
        'sawyer_joint_vel',
        'gripper_pos',
        'gripper_quat',
        'block_pose',
        'gripper_to_block',
    ], raise_on_missing=True))


# Let's add a reward for lifting the block up. We will have a 2-part reward -
# some of the reward comes from getting close to the block and some of the
# reward comes from actually lifting up the block. This shaping should help
# with learning a lift policy.
def distance_from_block_reward(
    observations: spec_utils.ObservationValue) -> float:
  gripper_to_block_distance = np.linalg.norm(
      observations[AddGripperToBlockObservation.OBS_KEY])
  return np.clip(1.0 - gripper_to_block_distance, 0, 1)

def lift_reward(observations: spec_utils.ObservationValue) -> float:
  obs_key = block_pose_sensor.get_obs_key(prop_pose_sensor.Observations.POSE)
  block_height = observations[obs_key][2]
  return np.clip(block_height * 2., 0, 1)

def sum_rewards(rewards: Sequence[float]) -> float:
  return np.clip(np.sum(rewards), 0, 1)

preprocessors.append(
    rewards.CombineRewards(
        reward_preprocessors=[
            rewards.ComputeReward(distance_from_block_reward),
            rewards.ComputeReward(lift_reward),
        ], combination_strategy=sum_rewards))


# Terminate the episodes after 5 sec. We are running at 10 Hz, so that means
# 50 environment steps.
preprocessors.append(subtask_termination.MaxStepsTermination(50))

## Make the environment

Great! Now we have all the components needed for an RL task:
 - a MuJoCo scene
 - the sensors and effectors to interact with the physical environment
 - task-specific logic in the form of action spaces, initializers, and timestep preprocessors

We can combine these into an RL environment by using MoMa's `SubtaskEnvironment`. The name "subtask" comes from Agentflow. Agentflow allows an agent to _flow_ from one task to the next, so each of the agent's objectives are really sub-tasks while the agent is completing a larger overarching goal.

However, in our case, we just have a single subtask that we would like to encapsulate as an RL environment. More specifically, we would like to adhere to DeepMind's canonical RL environment interface: `dm_env.Environment`. The MoMa `SubtaskEnvironment` helps do exactly that.

In [ ]:
# Add the initializer to the base task.
task.set_episode_initializer(entities_initializer)

env_builder = subtask_env_builder.SubtaskEnvBuilder()
env_builder.set_task(task)
env_builder.set_action_space(combined_action_space)
for preprocessor in preprocessors:
  env_builder.add_preprocessor(preprocessor)
environment = env_builder.build()

In [ ]:
# Let's take a look at the action, observation and reward specs.
print(f'Action spec: {environment.action_spec()}')
print('Observation spec:')
obs_spec = environment.observation_spec()
for key, spec in obs_spec.items():
  print(f'\t{key}: {spec}')
print(f'Reward spec: {environment.reward_spec()}')

# Run the environment

MoMa provides a `run_loop` to help step the agent and environment. It also provides useful tools for observing  the timesteps while pass to the agent.

In [ ]:
class RandomAgent:
  """An agent that emits uniform random actions."""

  def __init__(self, action_spec: specs.BoundedArray):
    self._spec = action_spec
    self._shape = action_spec.shape
    self._range = action_spec.maximum - action_spec.minimum

  def step(self, unused_timestep):
    action = (np.random.rand(*self._shape) - 0.5) * self._range
    np.clip(action, self._spec.minimum, self._spec.maximum, action)
    return action.astype(self._spec.dtype)

class CameraObserver:

  def __init__(self, physics_getter):
    self._physics_getter = physics_getter
    self._cam = None
    self._frames = []

  def begin_episode(self, *unused_args, **unused_kwargs):
    self._frames.append(render_scene(self._physics_getter()))

  def step(self, *unused_args, **unused_kwargs):
    self._frames.append(render_scene(self._physics_getter()))

  def end_episode(self, *unused_args, **unused_kwargs):
    # Add some black frames in between episodes.
    for _ in range(5):
      self._frames.append(np.zeros_like(self._frames[-1]))

  def get_frames(self):
    return self._frames

  def reset(self):
    self._frames = []


agent = RandomAgent(environment.action_spec())
observer = CameraObserver(lambda: environment.physics)

In [ ]:
# Run a couple episodes.
observer.reset()
run_loop.run(environment, agent, observers=[observer], max_steps=100)

# Render the video.
display_video(observer.get_frames(),
              framerate=int(1./environment.task.control_timestep))